In [1]:
# Terapkan capping campaign
import random
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
import joblib
from imblearn.over_sampling import RandomOverSampler

df_ml = pd.read_csv('bank-additional-full.csv', sep= ';')


X = df_ml.drop(columns=['y', 'duration'])
y = df_ml['y']

X_capped = X.copy()
campaign_cap = X_capped['campaign'].quantile(0.95)
X_capped['campaign'] = np.where(X_capped['campaign'] > campaign_cap, campaign_cap, X_capped['campaign'])

previous_cap = X_capped['previous'].quantile(0.95)
X_capped['previous'] = np.where(X_capped['previous'] > previous_cap, previous_cap, X_capped['previous'])

age_cap = X_capped['age'].quantile(0.99)
X_capped['age'] = np.where(X_capped['age'] > age_cap, age_cap, X_capped['age'])

X_capped['pdays_grouped'] = np.where(df_ml['pdays'] == 999, 'never_contacted', 'contacted_before')

# Drop kolom pdays asli jika sudah tidak digunakan
X_capped.drop(columns='pdays', inplace=True)

X_capped['job'] = X_capped['job'].replace('admin.', 'admin')

y_encoded = df_ml['y'].map({'no': 0, 'yes': 1})

X_train, X_test, y_train, y_test = train_test_split(
    X_capped, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


Final Model

In [3]:




num_cols = X_capped.select_dtypes(include='number').columns.tolist()
ordinal_cols = ['education']
nominal_cols = ['job', 'marital', 'default', 'month', 'day_of_week', 'housing', 'loan', 'contact', 'poutcome', 'pdays_grouped']

# === 2) Preprocessor dengan remainder='passthrough' ===
numeric_pipeline = RobustScaler()
ordinal_pipeline = OrdinalEncoder()
nominal_pipeline = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

from sklearn.metrics import confusion_matrix, make_scorer

def business_metric(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    total = tn + fp + fn + tp
    raw_score = ((tp*131 - (fp+tp)*18) - ((tp+fn)*131 - total*18)) / total
    # Normalisasi skor ke 0–1
    min_score = -14.52
    max_score = 18.0
    normalized_score = (raw_score - min_score) / (max_score - min_score)
    return normalized_score

business_score = make_scorer(business_metric, greater_is_better=True)

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_cols),
    ('ordinal', ordinal_pipeline, ordinal_cols),
    ('nominal', nominal_pipeline, nominal_cols)
], remainder='passthrough')

f2_scorer = make_scorer(fbeta_score, beta=2, pos_label=1)

from sklearn.model_selection import RandomizedSearchCV


lgbm_param = {
    'model__boosting_type': ['gbdt', 'dart'],
    'model__max_depth': [3, 5, 7, 10],
    'model__num_leaves': [20, 31, 50, 70],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__n_estimators': [100, 300, 500],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0],
    'model__reg_alpha': [0, 0.01, 0.1, 1, 10],
}

lgb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('RandomOverSampler', RandomOverSampler(random_state=42)),
    ('model', lgb.LGBMClassifier(random_state=0))
])

lgbm_search = RandomizedSearchCV(
    estimator=lgb_pipe,
    param_distributions=lgbm_param,
    n_iter=100,
    scoring=business_score,  # atau 'f2' jika kamu pakai make_scorer
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

lgbm_search.fit(X_train, y_train)
print("Best Params:", lgbm_search.best_params_)
print("Best Score:", lgbm_search.best_score_)

joblib.dump(lgbm_search, "model.sav")
print("✅ Model pipeline akhir berhasil disimpan!")

Fitting 5 folds for each of 100 candidates, totalling 500 fits


ValueError: 
All the 500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/redikru/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/redikru/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/redikru/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/Users/redikru/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 562, in _fit
    self._validate_steps()
  F

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler
import lightgbm as lgb

# Load data
df_ml = pd.read_csv('bank-additional-full.csv', sep=';')

# Prepare features and target
X = df_ml.drop(columns=['y', 'duration'])
y_encoded = df_ml['y'].map({'no': 0, 'yes': 1})

# Apply capping
X_capped = X.copy()
campaign_cap = X_capped['campaign'].quantile(0.95)
X_capped['campaign'] = np.where(X_capped['campaign'] > campaign_cap, campaign_cap, X_capped['campaign'])

previous_cap = X_capped['previous'].quantile(0.95)
X_capped['previous'] = np.where(X_capped['previous'] > previous_cap, previous_cap, X_capped['previous'])

age_cap = X_capped['age'].quantile(0.99)
X_capped['age'] = np.where(X_capped['age'] > age_cap, age_cap, X_capped['age'])

X_capped['pdays_grouped'] = np.where(df_ml['pdays'] == 999, 'never_contacted', 'contacted_before')
X_capped.drop(columns='pdays', inplace=True)
X_capped['job'] = X_capped['job'].replace('admin.', 'admin')

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_capped, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Define column types
num_cols = X_capped.select_dtypes(include='number').columns.tolist()
ordinal_cols = ['education']
nominal_cols = ['job', 'marital', 'default', 'month', 'day_of_week', 'housing', 'loan', 'contact', 'poutcome', 'pdays_grouped']

# Create preprocessor
numeric_pipeline = RobustScaler()
ordinal_pipeline = OrdinalEncoder()
nominal_pipeline = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_cols),
    ('ordinal', ordinal_pipeline, ordinal_cols),
    ('nominal', nominal_pipeline, nominal_cols)
], remainder='passthrough')

# Create pipeline with ImbPipeline
lgb_pipe = ImbPipeline([
    ('preprocessor', preprocessor),
    ('RandomOverSampler', RandomOverSampler(random_state=42)),
    ('model', lgb.LGBMClassifier(random_state=0))
])

# Test the pipeline
print("Testing pipeline...")
lgb_pipe.fit(X_train, y_train)
predictions = lgb_pipe.predict(X_test)
print(f"Pipeline works! Predictions shape: {predictions.shape}")
print(f"Number of positive predictions: {sum(predictions)}")
print("✅ Pipeline with RandomOverSampler is working correctly!")